In [42]:
from fastapi import FastAPI, Path, HTTPException, Query
from fastapi.responses import JSONResponse
from pydantic import BaseModel, Field, computed_field
from typing import Annotated, Literal, Optional
import json

app = FastAPI()


In [43]:
class Patient(BaseModel):

    id: Annotated[str, Field(..., description='ID of the patient', examples=['P001'])]
    name: Annotated[str, Field(..., description='Name of the patient')]
    city: Annotated[str, Field(..., description='City where the patient is living')]
    age: Annotated[int, Field(..., gt=0, lt=120, description='Age of the patient')]
    gender: Annotated[Literal['male', 'female', 'others'], Field(..., description='Gender of the patient')]
    height: Annotated[float, Field(..., gt=0, description='Height of the patient in mtrs')]
    weight: Annotated[float, Field(..., gt=0, description='Weight of the patient in kgs')]

    @computed_field
    @property
    def bmi(self) -> float:
        bmi = round(self.weight/(self.height**2),2)
        return bmi
    
    @computed_field
    @property
    def verdict(self) -> str:

        if self.bmi < 18.5:
            return 'Underweight'
        elif self.bmi < 25:
            return 'Normal'
        elif self.bmi < 30:
            return 'Normal'
        else:
            return 'Obese'


In [44]:
def load_data():
    with open('patients.json', 'r') as f:
        data = json.load(f)

    return data

def save_data(data):
    with open('patients.json', 'w') as f:
        json.dump(data, f)

In [45]:
load_data()

{'P001': {'name': 'Ananya Verma',
  'city': 'Guwahati',
  'age': 28,
  'gender': 'female',
  'height': 1.65,
  'weight': 90.0,
  'bmi': 33.06,
  'verdict': 'Obese'},
 'P002': {'name': 'Ravi Mehta',
  'city': 'Mumbai',
  'age': 35,
  'gender': 'male',
  'height': 1.75,
  'weight': 85,
  'bmi': 27.76,
  'verdict': 'Overweight'},
 'P003': {'name': 'Sneha Kulkarni',
  'city': 'Pune',
  'age': 22,
  'gender': 'female',
  'height': 1.6,
  'weight': 45,
  'bmi': 17.58,
  'verdict': 'Underweight'},
 'P004': {'name': 'Arjun Verma',
  'city': 'Mumbai',
  'age': 40,
  'gender': 'male',
  'height': 1.8,
  'weight': 90.0,
  'bmi': 27.78,
  'verdict': 'Normal'},
 'P005': {'name': 'Neha Sinha',
  'city': 'Kolkata',
  'age': 30,
  'gender': 'female',
  'height': 1.55,
  'weight': 75,
  'bmi': 31.22,
  'verdict': 'Obese'}}

In [46]:
@app.get("/")
def hello():
    return {'message':'Patient Management System API'}


In [47]:
a={
    'Ram':'20',
    'Vinod':'30'
}

In [48]:
for key,value in a.items():
    print(key,"->",value)

Ram -> 20
Vinod -> 30


In [49]:
a["Ram"]

'20'

In [50]:
@app.get('/patient/{patient_id}')
def view_patient(patient_id: str = Path(..., description='ID of the patient in the DB', examples='P001')):
    # load all the patients
    data = load_data()

    if patient_id in data:
        return data[patient_id]
    raise HTTPException(status_code=404, detail='Patient not found')


In [51]:
@app.post('/create')
def create_patient(patient: Patient):

    # load existing data
    data = load_data()

    # check if the patient already exists
    if patient.id in data:
        raise HTTPException(status_code=400, detail='Patient already exists')

    # new patient add to the database
    data[patient.id] = patient.model_dump(exclude=['id'])

    # save into the json file
    save_data(data)

    return JSONResponse(status_code=201, content={'message':'patient created successfully'})


In [54]:
import uvicorn
import nest_asyncio

# This allows uvicorn to run in Jupyter
nest_asyncio.apply()

# Run the server
uvicorn.run(app, host="localhost", port=8900, reload=False)


INFO:     Started server process [33428]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://localhost:8900 (Press CTRL+C to quit)


INFO:     ::1:50963 - "GET / HTTP/1.1" 200 OK
INFO:     ::1:50963 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     ::1:50964 - "GET /docs HTTP/1.1" 200 OK
INFO:     ::1:50964 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     ::1:50970 - "GET / HTTP/1.1" 200 OK
INFO:     ::1:50990 - "GET /patient/P001 HTTP/1.1" 200 OK
INFO:     ::1:51015 - "POST /create HTTP/1.1" 201 Created


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [33428]


In [ ]:
async def main():
    Database_URL = "Log_of_Events.db"
    db = DatabaseManager(Database_URL)

    # Create workflow instance
    workflow_instance = Workflow(db=db)
    app = await workflow_instance.Flow()
    
    # Dummy distress conversation transcript
    distress_transcript = """
    Woman: "Please stop following me, I don't know you"
    Unknown person: "Come on, don't be like that"
    Woman: "Don't touch me! Leave me alone!"
    Unknown person: "Just talk to me for a minute"
    Woman: "Help! Someone help me! Don't do this to me!"
    Woman: "I will call the police if you don't stop"
    Unknown person: "Nobody can hear you here"
    Woman: "Please don't hurt me, I haven't done anything"
    """
    with open("P.wav", "rb") as f:
         audio_b64 = base64.b64encode(f.read()).decode()
    
    # Create input state with all required fields
    input_state = WorkflowState(
        location=Location(latitude=12.9716, longitude=77.5946),  # Bangalore coordinates
        real_time=datetime.now(),  # 2:30 AM (night time)
        TAudio=None,
        threat_score=0,
        consent="Yes",
        gesture="Yes", 
        sudden_acc="Yes",
        message="Dangerous",
        audio_b64=audio_b64
    )
    
    # Run the workflow
    result = await app.ainvoke(input_state)

    print(type(result))
    print(result['analysis'])
    print(result['analysis'].threat_level)#result['analysis'] returns an object in return 

# Run the workflow
if __name__ == "__main__":
    await main()
